In [76]:
import numpy as np
import pandas 
import math 

In [77]:
#1. Загрузите данные из файла data-logistic.csv. 
data = pandas.read_csv('data-logistic.csv',header=None)
data[:2]

,0,1,2
0,-1,-0.663827,-0.138526
1,1,1.994596,2.468025


In [78]:
# Это двумерная выборка, целевая переменная на которой принимает значения -1 или 1.
y= data.loc[:,0]
y[:2]

0   -1
1    1
Name: 0, dtype: int64

In [79]:
#Здесь xi1 и xi2 — значение первого и второго признаков соответственно на объекте xi. 
x1= data.loc[:,1]
#x1[:2]
#x1

In [80]:
x2= data.loc[:,2]
x2[:2]

0   -0.138526
1    2.468025
Name: 2, dtype: float64

In [81]:
#2. Убедитесь, что выше выписаны правильные формулы для градиентного спуска. 
# Обратите внимание, что мы используем полноценный градиентный спуск, а не его стохастический вариант!

In [82]:
# Gradient     : w^{t+1} = w^{t} - h * d(Q(w))/dw = w^{t} - h * d(Sum^l_i(q_i(w))/dw
# Stochastic Gr: w^{t+1} = w^{t} - h * d(q_i(w))/dw - q_i - random number from sum
# -> in functionals w1 & w2 from task we have sums, therefore - gradient, not stochastic

In [118]:
def grad_step1(w1,w2,k,C,x1,x2,y):
    l=len(y)
    sum_q=0
    for i in range(1,l):
        sum_q+=y[i]*x1[i]*(1- 1/(1+np.exp(-y[i]*( w1*x1[i] + w2*x2[i] ) ) )  )
    
    #print (k,l)
    return w1+k*(1/l)*sum_q - k*C*w1

In [119]:
def grad_step2(w1,w2,k,C,x1,x2,y):
    l=len(y)
    sum_q=0
    for i in range(1,l):
        sum_q += y[i]*x2[i]*( 1 - 1/(1+math.exp(-y[i]*(w1*x1[i]+w2*x2[i]))) )
    return w2+k*(1/l)*sum_q - k*C*w2

In [120]:
# Q = 1/l sum L(w) =1/l sum log(1 +exp( -y_i(w1*x_i1+w2*x_i2))) +0.5*C|w^2|
# minimize Q wrt w1 and w2
# Градиентный шаг для весов будет заключаться в одновременном обновлении весов w1 и w2 

In [170]:
#3. Реализуйте градиентный спуск для обычной и L2-регуляризованной (с коэффициентом регуляризации 10) 
# логистической регрессии. Используйте длину шага k=0.1. В качестве начального приближения используйте вектор (0, 0).

def gradient_descent(y,x1,x2,C=0,w1=0,w2=0,k=0.1,precision=1e-5,N_max=10000):
    w1_plus1=w1
    w2_plus1=w2
    evkl_dist = 0
    #precision=1/N_max
    #Останавливать итерационный процесс можно, например, при близости градиента к нулю 
    #или при слишком малом изменении вектора весов на последней итерации. 
    for i in range(0,N_max):
        w1_plus1 = grad_step1(w1,w2,k,C,x1,x2,y)
        w2_plus1 = grad_step2(w1,w2,k,C,x1,x2,y)
        # евклидово расстояние между векторами весов на соседних итерациях должно быть не больше 1e-5
        evkl_dist = math.sqrt( (w1_plus1-w1)**2 + (w2_plus1-w2)**2 )
        if evkl_dist<=precision:
            print("The local minimum occurs at %0.f: w1=%0.3f , w2=%0.3f" % (i,w1_plus1,w2_plus1) )
            return [w1_plus1, w2_plus1]
            #break
        #w1=w1_plus1
        #w2=w2_plus1
        w1,w2=w1_plus1,w2_plus1
    #return [w1_plus1, w2_plus1]


In [171]:
#4. Запустите градиентный спуск и доведите до сходимости (евклидово расстояние между векторами весов на соседних 
# итерациях должно быть не больше 1e-5). Рекомендуется ограничить сверху число итераций десятью тысячами.


In [172]:
# обычной логистической регрессии
w1_n,w2_n=gradient_descent(y,x1,x2)
print (w1_n,w2_n)
#sum 1-l: 241 0.28436070521455276 0.0936185320546884
#sum 0-l: 243 0.2878116204717764 0.09198330215925439

The local minimum occurs at 241: w1=0.284 , w2=0.094
0.28436070521455276 0.0936185320546884


In [173]:
# L2-регуляризованной логистической регрессии
w1_l2r,w2_l2r=gradient_descent(y, x1,x2, 10)
print (w1_l2r,w2_l2r)
#0.2849621965007955 0.0930618620041445
#595 0.28840481937470264 0.09143492951108886
# c actually: The local minimum occurs at 7: w1=0.028 , w2=0.025 0.02841979291312511 0.024765289499758314

The local minimum occurs at 7: w1=0.028 , w2=0.025
0.02841979291312511 0.024765289499758314


In [174]:
#5. Какое значение принимает AUC-ROC(Area Under ROC-Curve) на обучении без регуляризации и при ее использовании? 
# Эти величины будут ответом на задание. В качестве ответа приведите два числа через пробел. 
# что на вход функции roc_auc_score нужно подавать оценки вероятностей, подсчитанные обученным алгоритмом. 
# Для этого воспользуйтесь сигмоидной функцией: a(x) = 1 / (1 + exp(-w1 x1 - w2 x2)).
from  sklearn.metrics import roc_auc_score
#x1[204]

In [175]:
#roc_auc_score(y_true, y_score, average=’macro’, sample_weight=None)
#y_score - a(x) = 1 / (1 + exp(-w1 x1 - w2 x2)).
y_n = []
y_l2r = []
for i in range(0,len(y)):
    #print (i)
    y_n.append(1 / (1+np.exp( -w1_n*x1[i] - w2_n*x2[i]) ) )
    y_l2r.append(1 / (1+np.exp(-w1_l2r*x1[i]-w2_l2r*x2[i])) )


In [176]:
#len(y_l2r)
#y_l2r

In [177]:
AUCROC_n = roc_auc_score(y,y_n)


In [178]:
AUCROC_l2r = roc_auc_score(y,y_l2r)
print (AUCROC_n, AUCROC_l2r)

0.9270476190476189 0.9362857142857142


In [179]:
print ("nom %0.3f, reg %0.3f" % (AUCROC_n, AUCROC_l2r))

nom 0.927, reg 0.936
